## How to use 


### What you need to run the API

   * An Agrimetrics subscription key
   * An SBI

### Steps

   * Import shapes
   * Get shapes, filtering by type `RPA`
   

### Install dependencies

In [ ]:
import sys
!{sys.executable} -m pip install shapely geopandas matplotlib --quiet

### Set SBI number

In [ ]:
import os
from getpass import getpass
from shapely.geometry import mapping, shape as shapelyShape

if "SBI_KEY" in os.environ:
    sbi = os.environ["SBI_KEY"]
else:
    sbi = getpass(prompt="Enter your SBI:").strip()

### Set API key

In [ ]:
if "API_KEY" in os.environ:
    subscription_key = os.environ["API_KEY"]
else:
    subscription_key = getpass(prompt="Enter your subscription key:").strip()

### Import RPA shapes

In [ ]:
import requests
import json

import_shapes = input(
    "Do you want to import your RPA field boundaries from DEFRA? y/n:"
)

if import_shapes.lower() == "y":

    url = "https://api-test.agrimetrics.co.uk/shapes/services/rpa/import"

    payload = json.dumps({"sbi": sbi})
    agrimetrics_headers = {
        "ocp-apim-subscription-key": subscription_key,
        "Content-Type": "application/json",
    }

    response = requests.post(url, headers=agrimetrics_headers, data=payload)

    if response.ok:
        message = json.loads(response.text).get("message", "0 shapes created")
        number_of_shapes = int(message.split()[0])
        print(
            f"{number_of_shapes} RPA boundaries imported to the Agrimetrics Shapes API"
        )

### Get RPA boundaries from Agrimetrics Shapes API

In [ ]:
all_shapes = []

max_pages = 10
results_per_page = 100  # note there is a maximum page size of 100

headers = agrimetrics_headers


def get_shapes(page=1, results=results_per_page):
    shapes_url = f"https://api-test.agrimetrics.co.uk/shapes/shapes?pageNum={page}&pageSize={results}"
    shapes_response = requests.get(shapes_url, headers=agrimetrics_headers).json()
    print(shapes_response)
    return shapes_response["shapes"]


for page in range(1, max_pages + 1):
    shapes = get_shapes(page)
    if not shapes:
        break
    all_shapes.extend(shapes)

rpa_shapes = [shape for shape in all_shapes if shape["source"] == "rpa"]

### Get layers for data set from Agrimetrics Geospatial API

In [ ]:
import requests

dataset_id = "f0985f68-3b10-4d67-b0f2-3d8521e38dec"


def getLayersFromDataset(dataset=dataset_id):
    url = f"https://api-test.agrimetrics.co.uk/geospatial/datasets/{dataset_id}"
    response = requests.get(url, headers=agrimetrics_headers).json()
    layer_in_response = [layer["links"]["self"] for layer in response["layers"]]
    return layer_in_response


layerIds = getLayersFromDataset()
print(f"Working with the following layers: {layerIds}")

### Search shapes against data set

Search Geospatial API for each layer in each RPA shape. The shape is first simplified to help search performance.

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

SHAPE_TOLERANCE = 0.05  # lower value means less simplification
NUMBER_OF_RPA_BOUNDARIES_TO_CHECK = 10


def searchGeospatial(boundary, layer):
    geospatial_url = f"{layer}/search"

    simplified = shapelyShape(boundary).simplify(
        SHAPE_TOLERANCE, preserve_topology=True
    )

    body = json.dumps(
        {"geometry": mapping(simplified), "outputFormat": "application/json"}
    )

    return requests.post(geospatial_url, headers=agrimetrics_headers, data=body)


for shape in rpa_shapes[:NUMBER_OF_RPA_BOUNDARIES_TO_CHECK]:

    full_shape = requests.get(shape["id"], headers=agrimetrics_headers).json()
    searchResponse = searchGeospatial(
        boundary=full_shape["boundary"], layer=layerIds[0]
    )
    output = searchResponse.json()

    rpa_feature = {
        "type": "Feature",
        "id": "temp_id",
        "geometry": full_shape["boundary"],
        "properties": {},
    }

    df_rpa_boundary = gpd.GeoDataFrame.from_features([rpa_feature])
    df_water_catchments = gpd.GeoDataFrame.from_features(output.get("features", []))

    print("--------------------------------------------------")

    print(f"RPA field id: {shape['userLabel']} is in the following water catchment(s):")
    water_plot = df_water_catchments.plot("wb_name", cmap="plasma")
    df_rpa_boundary.plot(ax=water_plot.axes, cmap="spring")
    plt.show()
    for feature in output.get("features", []):
        print(f"     {feature['properties']['wb_name']}")